# Data Filtering

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_M = pd.read_csv('datos_analisis_por_comuna.csv') 
df_M = df_M.loc[:, ~df_M.columns.str.contains('^Unnamed')]
df_M.head()

,created_on,end_date,delta_time,lon,lat,bedrooms,bathrooms,surface_total,price,imput,...,MANRIQUE,PALMITAS,POPULAR,ROBLEDO,SAN ANTONIO DE PRADO,SAN CRISTOBAL,SAN JAVIER,SANTA CRUZ,SANTA ELENA,VILLA HERMOSA
0,9/14/2018,9/25/2018,11,-75.577025,6.211360,NaN,NaN,134,19.519293,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9/14/2018,9/25/2018,11,-75.572928,6.234350,NaN,NaN,588,19.701615,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9/14/2018,9/25/2018,11,-75.553186,6.237236,NaN,NaN,160,20.125429,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9/14/2018,9/25/2018,11,-75.594599,6.226078,NaN,NaN,90,19.583832,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9/14/2018,9/25/2018,11,-75.542267,6.205542,NaN,NaN,101,19.781657,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [3]:
# Separating the real values (not imputed)
df_real = df_M.loc[df_M['imput'] == 0]
print(df_real.shape)

# Counting NaN values of bedrooms and bathrooms respectively
print('Bedrooms nan values: ', df_real['bedrooms'].isna().sum())
print('Bathrooms nan values: ', df_real['bathrooms'].isna().sum())

(18775, 40)
Bedrooms nan values:  6199
Bathrooms nan values:  853


In [4]:
# Eliminate all the rows that contain a NaN value (provisional)
df_real = df_real.dropna(axis = 0)
df_real.shape

(12537, 40)

In [5]:
# Selection of households and apartments
esCasaoApto = df_real[['Apartamento', 'Casa']].apply(lambda x: False if (x[0] == 0 and x[1] == 0) else True, axis = 1)
df_real = df_real.loc[esCasaoApto].drop(columns = ['imput', 'Casa', 'Depósito', 'Finca', 'Local comercial', 'Lote', 
                                                   'Parqueadero', 'Oficina', 'Otro'])
df_real.columns

Index(['created_on', 'end_date', 'delta_time', 'lon', 'lat', 'bedrooms',
       'bathrooms', 'surface_total', 'price', 'Apartamento', 'ALTAVISTA',
       'ARANJUEZ', 'BELEN', 'BUENOS AIRES', 'CASTILLA', 'DOCE DE OCTUBRE',
       'EL POBLADO', 'GUAYABAL', 'LA AMERICA', 'LA CANDELARIA', 'LAURELES',
       'MANRIQUE', 'PALMITAS', 'POPULAR', 'ROBLEDO', 'SAN ANTONIO DE PRADO',
       'SAN CRISTOBAL', 'SAN JAVIER', 'SANTA CRUZ', 'SANTA ELENA',
       'VILLA HERMOSA'],
      dtype='object')

In [6]:
df_real['price'] = df_real['price'].apply(lambda x: np.exp(x))

# Comparing basic traits of each district

In [7]:
# Extracting the names of every district
columnas = df_real.columns.to_series()
nombres = columnas.loc['ALTAVISTA':]
nombres.reset_index(inplace = True, drop = True)
nombres

0                ALTAVISTA
1                 ARANJUEZ
2                    BELEN
3             BUENOS AIRES
4                 CASTILLA
5          DOCE DE OCTUBRE
6               EL POBLADO
7                 GUAYABAL
8               LA AMERICA
9            LA CANDELARIA
10                LAURELES
11                MANRIQUE
12                PALMITAS
13                 POPULAR
14                 ROBLEDO
15    SAN ANTONIO DE PRADO
16           SAN CRISTOBAL
17              SAN JAVIER
18              SANTA CRUZ
19             SANTA ELENA
20           VILLA HERMOSA
dtype: object

In [8]:
# Creating a list of dataframes containing the information of district
districts_df = [None] * nombres.shape[0]
d = {}
i = 0
for district in nombres:
    name = district.replace(' ', '_')
    vars()[name] = df_real.loc[df_real[district] == 1].loc[:,:'price'] 
    districts_df[i] = vars()[name]
    d[i] = name
    i += 1

In [9]:
# Describing each data set and extracting the data and storing it in a list
delta_time = [None] * nombres.shape[0]
bedrooms = [None] * nombres.shape[0]
bathrooms = [None] * nombres.shape[0]
surface_total = [None] * nombres.shape[0]
price = [None] * nombres.shape[0]
size = [None] * nombres.shape[0]

i = 0

while i < nombres.shape[0]:
    df = districts_df[i].describe().loc['mean'].drop(['lat', 'lon'])
    delta_time[i] = df[0]
    bedrooms[i] = df[1]
    bathrooms[i] = df[2]
    surface_total[i] = df[3]
    price[i] = df[4]
    size[i] = districts_df[i].shape[0]
    i += 1

In [10]:
# Turning it into a unique dataframe
districts = pd.DataFrame({
    'name': nombres.values.tolist(),
    'delta_time': delta_time, 
    'bedrooms': bedrooms, 
    'bathrooms': bathrooms, 
    'surface_total': surface_total,
    'price': price,
    'size': size
})
districts

,name,delta_time,bedrooms,bathrooms,surface_total,price,size
0,ALTAVISTA,68.730769,2.750000,2.057692,84.596154,3.468077e+08,52
1,ARANJUEZ,68.839080,4.068966,2.275862,149.310345,3.375081e+08,87
2,BELEN,66.717514,3.378531,2.674200,137.879473,4.365178e+08,1062
3,BUENOS AIRES,87.429213,3.804494,2.997753,278.516854,6.795728e+08,445
4,CASTILLA,72.098039,3.705882,2.568627,130.647059,3.837451e+08,51
5,DOCE DE OCTUBRE,36.400000,3.600000,3.000000,136.200000,4.380000e+08,5
6,EL POBLADO,75.208403,3.297059,3.497059,386.276891,8.266809e+08,2380
7,GUAYABAL,79.724280,3.438272,2.823045,201.629630,5.938528e+08,486
8,LA AMERICA,69.788686,3.723794,2.607321,143.811980,3.978516e+08,601
9,LA CANDELARIA,59.983752,3.691285,2.927622,176.428360,5.508242e+08,677


In [71]:
ordenados_por_tamano = districts.sort_values(['size'])
ordenados_por_tamano

,name,delta_time,bedrooms,bathrooms,surface_total,price,size
18,SANTA CRUZ,37.000000,3.000000,2.000000,67.000000,2.400000e+08,2
5,DOCE DE OCTUBRE,36.400000,3.600000,3.000000,136.200000,4.380000e+08,5
12,PALMITAS,119.600000,3.400000,2.800000,118.600000,3.840000e+08,5
13,POPULAR,55.000000,4.636364,2.818182,213.727273,4.277273e+08,11
15,SAN ANTONIO DE PRADO,104.833333,3.750000,3.000000,255.083333,6.700000e+08,12
16,SAN CRISTOBAL,115.043478,3.739130,2.869565,148.739130,3.903478e+08,23
17,SAN JAVIER,72.934783,3.782609,2.673913,282.456522,4.119130e+08,46
4,CASTILLA,72.098039,3.705882,2.568627,130.647059,3.837451e+08,51
11,MANRIQUE,74.294118,4.000000,2.627451,178.490196,4.564902e+08,51
0,ALTAVISTA,68.730769,2.750000,2.057692,84.596154,3.468077e+08,52


# Plotting

In [11]:
import plotly
from plotly.offline import init_notebook_mode, plot, iplot, download_plotlyjs
init_notebook_mode(connected = True)

import plotly.graph_objs as go

In [12]:
trace = go.Bar(x = districts.sort_values(['price']).name, 
               y = districts.sort_values(['price']).price.apply(lambda x: x/1000000))
layout = go.Layout(title = 'Precio Medio por Comuna', xaxis_title = 'Comuna', 
                   yaxis_title = 'Precio COP en (M)')
fig = go.Figure(data = [trace], layout = layout)
iplot(fig)

In [13]:
trace = go.Bar(x = districts.sort_values(['surface_total']).name, 
               y = districts.sort_values(['surface_total']).surface_total)
layout = go.Layout(title = 'Superfcie Total Media por Comuna', xaxis_title = 'Comuna', 
                   yaxis_title = 'Superficie Total (m^2)')
fig = go.Figure(data = [trace], layout = layout)
iplot(fig)

In [14]:
trace1 = go.Bar(x = districts.sort_values(['price']).name, 
               y = districts.sort_values(['price']).price.apply(lambda x: x/1000000),
               name = 'Precio COP (M)', marker_color = '#0b31db')
trace = go.Bar(x = districts.sort_values(['surface_total']).name, 
                y = districts.sort_values(['surface_total']).surface_total,
                name = 'Superficie Total', marker_color = '#db700b')
layout = go.Layout(title = 'Precio y Superficie total Media por Comuna', xaxis_title = 'Comuna', barmode = 'group')
fig = go.Figure(data = [trace, trace1], layout = layout)
fig.add_trace(go.Scatter(x = districts.sort_values(['surface_total']).name, 
                         y = districts.sort_values(['surface_total']).price.apply(lambda x: x/1000000),
                         mode = 'lines+markers',
                         name = 'Precio COP (M)'))
fig.add_trace(go.Scatter(x = districts.sort_values(['surface_total']).name,
                         y = districts.sort_values(['surface_total']).surface_total,
                         mode = 'lines+markers',
                         name = 'Superficie Total'))
iplot(fig)

In [15]:
trace = go.Bar(x = districts.sort_values(['bedrooms']).name, y = districts.sort_values(['bedrooms']).bedrooms)
layout = go.Layout(title = 'Numero Medio de Cuartos por Comuna', xaxis_title = 'Comuna', yaxis_title = 'Cuartos')
fig = go.Figure(data = [trace], layout = layout)
iplot(fig)

In [16]:
trace = go.Bar(x = districts.sort_values(['bathrooms']).name, y = districts.sort_values(['bathrooms']).bathrooms)
layout = go.Layout(title = 'Numero Medio de Baños por Comuna', xaxis_title = 'Comuna', yaxis_title = 'Baños')
fig = go.Figure(data = [trace], layout = layout)
iplot(fig)

In [17]:
trace1 = go.Bar(x = districts.sort_values(['bathrooms']).name, 
                y = districts.sort_values(['bathrooms']).bathrooms, name = 'Baños', marker_color = '#db700b')
trace = go.Bar(x = districts.sort_values(['bathrooms']).name, 
               y = districts.sort_values(['bathrooms']).bedrooms, name = 'Cuartos')
layout = go.Layout(title = 'Numero de Medio de Baños y Cuartos (Orden por Precio)', 
                   xaxis_title = 'Comuna', barmode = 'group')
fig = go.Figure(data = [trace, trace1], layout = layout)
fig.add_trace(go.Scatter(x = districts.sort_values(['bathrooms']).name, 
                         y = districts.sort_values(['bathrooms']).bathrooms,
                         mode = 'lines+markers',
                         name = 'Baños'))
fig.add_trace(go.Scatter(x = districts.sort_values(['bathrooms']).name,
                         y = districts.sort_values(['bathrooms']).bedrooms,
                         mode = 'lines+markers',
                         name = 'Cuartos'))
fig.add_trace(go.Scatter(x = districts.sort_values(['bathrooms']).name,
                         y = districts.sort_values(['bathrooms']).price.apply(lambda x: x/100000000),
                         mode = 'lines+markers',
                         name = 'Precio COP (100M)',
                         marker_color = 'black'))
iplot(fig)

# Observaciones
 
## Superficie total
 - Es probable que las viviendas que sean catalogadas como casas en Santa Elena, en realidad sean fincas, debido a su alto precio medio y su alta suferpeficie total media.
 - Hay comunas en las cuales la superficie total de no sigue la misma tendencia que el precio necesariamente. Esto puede ocurrir debido a que la zona efectivamente infla los precios de las viviendas que se encuentrar en dichos distritos, y los precios en esta zona efectivamente son delimitados por su sector.
 - Los picos de la anomalia mencionada previamentre corresponden a las comunas: LAURELES, LA CANDELARIA, GUAYABAL, VILLA HERMOSA, SAN ANTONIO DE PRADO, EL POBLADO y SANTA ELENA.
 - Los bajos corresponden a: ARANJUEZ, MANRIQUE, POPULAR y SAN JAVIER.

## Cuartos y Baños
 - El precio tampoco sigue la misma tendencia del numero medio de cuartos y baños por comuna. 
 - Cuando se organiza con respecto a los baños, hay picos en: VILLA HERMOSA, GUAYABAL, LA CANDELARIA, LAURELES, BUENOS AIRES, SAN ANTONIO DE PRADO, EL POBLADO Y SANTA ELENA.
 - Los bajos corresponden a: PALMITAS, POPULAR, SAN CRISTOBAL, Y EL DOCE DE OCTUBRE. 
 - Cuando se organizan con respecto al numero medio de cuartos el precio no sigue una tendencia clara en ninguna parte de la grafica, a diferencia de las graficas de las superficie total media y el numero medio de baños, donde habia una clara relacion entre estas dos variables al principio de la grafica. 

# Analisis de Picos en Superficie Total

## Santa Elena

In [25]:
# Filtering
santa_elena = df_real.loc[df_real['SANTA ELENA'] == 1].drop(columns = ['ALTAVISTA', 'ARANJUEZ', 'BELEN', 
                                                                       'BUENOS AIRES','CASTILLA', 'DOCE DE OCTUBRE',
                                                                       'EL POBLADO', 
                                                                       'GUAYABAL', 'LA AMERICA', 'LA CANDELARIA', 
                                                                       'LAURELES', 'PALMITAS', 'POPULAR', 'ROBLEDO', 
                                                                       'MANRIQUE', 'SAN ANTONIO DE PRADO',
                                                                       'SAN CRISTOBAL', 
                                                                       'SAN JAVIER', 'SANTA CRUZ', 'SANTA ELENA',
                                                                       'VILLA HERMOSA'])
santa_elena.describe()

,delta_time,lon,lat,bedrooms,bathrooms,surface_total,price,Apartamento
count,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,8.100000e+01,81.000000
mean,75.555556,-75.551486,6.202292,3.864198,5.086420,627.209877,1.852837e+09,0.123457
std,62.424955,0.007957,0.016980,0.862454,1.797203,600.356909,1.140980e+09,0.331010
min,3.000000,-75.559286,6.179083,2.000000,1.000000,96.000000,2.800000e+08,0.000000
25%,32.000000,-75.555849,6.187518,3.000000,4.000000,237.000000,7.800000e+08,0.000000
50%,76.000000,-75.552002,6.197128,4.000000,5.000000,496.000000,1.700000e+09,0.000000
75%,91.000000,-75.547997,6.224320,4.000000,7.000000,700.000000,2.700000e+09,0.000000
max,364.000000,-75.498001,6.226688,6.000000,8.000000,2700.000000,4.300000e+09,1.000000


In [26]:
apto_elena = santa_elena[santa_elena['Apartamento'] == 1]
casa_elena = santa_elena[santa_elena['Apartamento'] == 0]
print('Numero de apatamentos: ', apto_elena.shape[0])
print('Numero de casas: ', casa_elena.shape[0])

Numero de apatamentos:  10
Numero de casas:  71


In [27]:
trace1 = go.Scatter(x = apto_elena.surface_total, y = apto_elena.price, mode = 'markers', name = 'Apartamentos',
                   marker_color = '#ffd000')
trace2 = go.Scatter(x = casa_elena.surface_total, y = casa_elena.price, mode = 'markers', name = 'Casas', 
                   marker_color = '#1caeb0')
layout = go.Layout(title = 'Superficie Total vs Precio (SANTA ELENA)',
                   xaxis_title = 'Superficie Total',
                   yaxis_title = 'Precio COP')
fig = go.Figure(data = [trace1, trace2], layout =  layout)
iplot(fig)

## Observaciones 

##### Primeras Observaciones:
 - Claramente la superficie total de incrementa de manera notoria el precio de las viviendas.
 - Si se compara con la superficie total de EL POBLADO (Esto ya no es valido o seguro, debido a que se descubrio que hay un outlier que incrementa el valor de la superficie total media), comuna cuya superficie total es la segunda mayor, despues de esta (SANTA ELENA), en general, las superficies totales son bastante grandes, lo que puede indicar que efectivamemnte la mayoria de estas viviendas pueden ser fincas
 - Es sorprendente que los apartamentos tengan una superficie tan grande, en especial por la zona en la que se esta, puede que estos tambien esten mal catalogados y en realidad sean casas finca.

## Laureles

In [23]:
# Filtering
laureles = df_real.loc[df_real['LAURELES'] == 1].drop(columns = ['ALTAVISTA', 'ARANJUEZ', 'BELEN', 
                                                                       'BUENOS AIRES','CASTILLA', 'DOCE DE OCTUBRE',
                                                                       'EL POBLADO', 
                                                                       'GUAYABAL', 'LA AMERICA', 'LA CANDELARIA', 
                                                                       'LAURELES', 'PALMITAS', 'POPULAR', 'ROBLEDO', 
                                                                       'MANRIQUE', 'SAN ANTONIO DE PRADO',
                                                                       'SAN CRISTOBAL', 
                                                                       'SAN JAVIER', 'SANTA CRUZ', 'SANTA ELENA',
                                                                       'VILLA HERMOSA'])
laureles.describe()

,delta_time,lon,lat,bedrooms,bathrooms,surface_total,price,Apartamento
count,2349.000000,2349.000000,2349.000000,2349.000000,2349.000000,2349.000000,2.349000e+03,2349.000000
mean,62.255002,-75.593307,6.247360,3.632184,2.960409,172.836100,5.439884e+08,0.564070
std,56.034723,0.007841,0.007116,1.365835,1.207465,211.890149,4.151477e+08,0.495984
min,0.000000,-75.610000,6.237556,1.000000,1.000000,11.000000,2.400000e+08,0.000000
25%,18.000000,-75.601334,6.243000,3.000000,2.000000,95.000000,3.270000e+08,0.000000
50%,56.000000,-75.594002,6.244705,3.000000,3.000000,130.000000,4.250000e+08,1.000000
75%,89.000000,-75.586998,6.249000,4.000000,4.000000,200.000000,5.800000e+08,1.000000
max,460.000000,-75.578003,6.270000,18.000000,10.000000,5700.000000,4.500000e+09,1.000000


In [24]:
apto_laureles = laureles[laureles['Apartamento'] == 1]
casa_laureles = laureles[laureles['Apartamento'] == 0]
print('Numero de apatamentos: ', apto_laureles.shape[0])
print('Numero de casas: ', casa_laureles.shape[0])

Numero de apatamentos:  1325
Numero de casas:  1024


In [29]:
trace1 = go.Scatter(x = apto_laureles.surface_total, y = apto_laureles.price, mode = 'markers', name = 'Apartamentos',
                   marker_color = '#ffd000')
trace2 = go.Scatter(x = casa_laureles.surface_total, y = casa_laureles.price, mode = 'markers', name = 'Casas', 
                   marker_color = '#1caeb0')
layout = go.Layout(title = 'Superficie Total vs Precio (LAURELES)',
                   xaxis_title = 'Superficie Total',
                   yaxis_title = 'Precio COP')
fig = go.Figure(data = [trace1, trace2], layout =  layout)
iplot(fig)

## Observaciones

###### Primeras Observaciones:
 - El conjunto de datos es lo suficientemente grande como para ser representativo, y es bastante balanceado con respecto al tipo de vivienda.
 - Existen datos outliers que evidentemente estan moviendo la media de su lugar, los que respectan a la superficie total pueden ser datos mal catalogados o digitados, y los correspondientes al precio pueden ser lo mismo, o por el contrario casas bastante grandes (esto lo creo posible dado el sector). 
 - Hay dos datos outliers con respecto a los apartamentos tambien, uno claramente es un dato mal catalogado, y otro puede ser una mala digitacion, o un apartamento excesivamente caro.

## La Candelaria

In [30]:
# Filtering
candelaria = df_real.loc[df_real['LA CANDELARIA'] == 1].drop(columns = ['ALTAVISTA', 'ARANJUEZ', 'BELEN', 
                                                                       'BUENOS AIRES','CASTILLA', 'DOCE DE OCTUBRE',
                                                                       'EL POBLADO', 
                                                                       'GUAYABAL', 'LA AMERICA', 'LA CANDELARIA', 
                                                                       'LAURELES', 'PALMITAS', 'POPULAR', 'ROBLEDO', 
                                                                       'MANRIQUE', 'SAN ANTONIO DE PRADO',
                                                                       'SAN CRISTOBAL', 
                                                                       'SAN JAVIER', 'SANTA CRUZ', 'SANTA ELENA',
                                                                       'VILLA HERMOSA'])
candelaria.describe()

,delta_time,lon,lat,bedrooms,bathrooms,surface_total,price,Apartamento
count,677.000000,677.000000,677.000000,677.000000,677.000000,677.000000,6.770000e+02,677.000000
mean,59.983752,-75.567574,6.247760,3.691285,2.927622,176.428360,5.508242e+08,0.621861
std,50.180338,0.005167,0.007869,1.531947,1.249529,217.496738,6.140060e+08,0.485281
min,0.000000,-75.578003,6.224662,1.000000,1.000000,40.000000,2.370000e+08,0.000000
25%,15.000000,-75.571999,6.246000,3.000000,2.000000,87.000000,2.950000e+08,0.000000
50%,61.000000,-75.566002,6.248000,3.000000,3.000000,125.000000,3.800000e+08,1.000000
75%,89.000000,-75.565000,6.253000,4.000000,3.000000,190.000000,5.200000e+08,1.000000
max,299.000000,-75.556508,6.265000,13.000000,10.000000,2710.000000,4.600000e+09,1.000000


In [35]:
apto_candelaria = candelaria[candelaria['Apartamento'] == 1]
casa_candelaria = candelaria[candelaria['Apartamento'] == 0]
print('Numero de apatamentos: ', apto_candelaria.shape[0])
print('Numero de casas: ', casa_candelaria.shape[0])

Numero de apatamentos:  421
Numero de casas:  256


In [38]:
trace1 = go.Scatter(x = apto_candelaria.surface_total, y = apto_candelaria.price, mode = 'markers', 
                    name = 'Apartamentos', marker_color = '#ffd000')
trace2 = go.Scatter(x = casa_candelaria.surface_total, y = casa_candelaria.price, mode = 'markers', name = 'Casas', 
                   marker_color = '#1caeb0')
layout = go.Layout(title = 'Superficie Total vs Precio (LA CANDELARIA)',
                   xaxis_title = 'Superficie Total',
                   yaxis_title = 'Precio COP')
fig = go.Figure(data = [trace1, trace2], layout =  layout)
iplot(fig)

## Observaciones

##### Primeras Observaciones:
 - Es un conjunto de datos relaticamente pequeño, sin embargo, dado la clara agrupacion en la parte inferior izquierda, puede que estos datos lleguen a ser representativos. 
 - Evidentemente hay datos de outliers de apartamentos y casas, uen caso de los apartamentos su superficie indica que estan mal catalogados o digitados. Asi mismo, el caso de las casas puede ser el mismo

## Villa hermosa

In [33]:
# Filtering
villa_hermosa = df_real.loc[df_real['VILLA HERMOSA'] == 1].drop(columns = ['ALTAVISTA', 'ARANJUEZ', 'BELEN', 
                                                                       'BUENOS AIRES','CASTILLA', 'DOCE DE OCTUBRE',
                                                                       'EL POBLADO', 
                                                                       'GUAYABAL', 'LA AMERICA', 'LA CANDELARIA', 
                                                                       'LAURELES', 'PALMITAS', 'POPULAR', 'ROBLEDO', 
                                                                       'MANRIQUE', 'SAN ANTONIO DE PRADO',
                                                                       'SAN CRISTOBAL', 
                                                                       'SAN JAVIER', 'SANTA CRUZ', 'SANTA ELENA',
                                                                       'VILLA HERMOSA'])
villa_hermosa.describe()

,delta_time,lon,lat,bedrooms,bathrooms,surface_total,price,Apartamento
count,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,7.600000e+01,76.000000
mean,62.736842,-75.550172,6.250035,4.223684,2.815789,208.342105,6.101053e+08,0.421053
std,58.639092,0.007211,0.007580,2.017250,1.162876,162.330819,7.971904e+08,0.497009
min,0.000000,-75.558998,6.232000,2.000000,2.000000,62.000000,2.400000e+08,0.000000
25%,17.750000,-75.555000,6.245394,3.000000,2.000000,89.250000,2.775000e+08,0.000000
50%,55.500000,-75.552002,6.251000,4.000000,2.000000,142.500000,3.200000e+08,0.000000
75%,89.000000,-75.544692,6.257128,4.000000,3.000000,255.000000,4.825000e+08,1.000000
max,299.000000,-75.530000,6.260000,12.000000,6.000000,680.000000,3.500000e+09,1.000000


In [37]:
apto_villa = villa_hermosa[villa_hermosa['Apartamento'] == 1]
casa_villa = villa_hermosa[villa_hermosa['Apartamento'] == 0]
print('Numero de apatamentos: ', apto_villa.shape[0])
print('Numero de casas: ', casa_villa.shape[0])

Numero de apatamentos:  32
Numero de casas:  44


In [39]:
trace1 = go.Scatter(x = apto_villa.surface_total, y = apto_villa.price, mode = 'markers', 
                    name = 'Apartamentos', marker_color = '#ffd000')
trace2 = go.Scatter(x = casa_villa.surface_total, y = casa_villa.price, mode = 'markers', name = 'Casas', 
                   marker_color = '#1caeb0')
layout = go.Layout(title = 'Superficie Total vs Precio (VILLA HERMOSA)',
                   xaxis_title = 'Superficie Total',
                   yaxis_title = 'Precio COP')
fig = go.Figure(data = [trace1, trace2], layout =  layout)
iplot(fig)

## Observaciones

##### Primeras Observaciones:
 - Al ser tan pequeño, y los datos estar tan esparcidos. No estoy seguro si es conjunto de datos es representativo. INVESTIGAR. 
 - Conjuntos de tipos de viviendas estan balanceados
 - Dos datos outliers, no estaria seguro de afirmar si son casas fincas o no. 

## San Antonio de Prado

In [40]:
# Filtering
san_antonio = df_real.loc[df_real['SAN ANTONIO DE PRADO'] == 1].drop(columns = ['ALTAVISTA', 'ARANJUEZ', 'BELEN', 
                                                                       'BUENOS AIRES','CASTILLA', 'DOCE DE OCTUBRE',
                                                                       'EL POBLADO', 
                                                                       'GUAYABAL', 'LA AMERICA', 'LA CANDELARIA', 
                                                                       'LAURELES', 'PALMITAS', 'POPULAR', 'ROBLEDO', 
                                                                       'MANRIQUE', 'SAN ANTONIO DE PRADO',
                                                                       'SAN CRISTOBAL', 
                                                                       'SAN JAVIER', 'SANTA CRUZ', 'SANTA ELENA',
                                                                       'VILLA HERMOSA'])
san_antonio.describe()

,delta_time,lon,lat,bedrooms,bathrooms,surface_total,price,Apartamento
count,12.000000,12.000000,12.000000,12.00000,12.0000,12.000000,1.200000e+01,12.000000
mean,104.833333,-75.661065,6.196801,3.75000,3.0000,255.083333,6.700000e+08,0.500000
std,79.916813,0.011247,0.011345,1.05529,1.3484,347.258996,6.581379e+08,0.522233
min,13.000000,-75.683998,6.175000,2.00000,1.0000,69.000000,2.800000e+08,0.000000
25%,64.500000,-75.665459,6.187250,3.75000,2.0000,96.750000,3.300000e+08,0.000000
50%,84.000000,-75.665333,6.204028,4.00000,3.5000,121.000000,4.150000e+08,0.500000
75%,125.750000,-75.654999,6.204304,4.00000,4.0000,149.500000,5.550000e+08,1.000000
max,299.000000,-75.640000,6.210000,6.00000,5.0000,1200.000000,2.500000e+09,1.000000


In [41]:
apto_santonio = san_antonio[san_antonio['Apartamento'] == 1]
casa_santonio = san_antonio[san_antonio['Apartamento'] == 0]
print('Numero de apatamentos: ', apto_santonio.shape[0])
print('Numero de casas: ', casa_santonio.shape[0])

Numero de apatamentos:  6
Numero de casas:  6


In [43]:
trace1 = go.Scatter(x = apto_santonio.surface_total, y = apto_santonio.price, mode = 'markers', 
                    name = 'Apartamentos', marker_color = '#ffd000')
trace2 = go.Scatter(x = casa_santonio.surface_total, y = casa_santonio.price, mode = 'markers', name = 'Casas', 
                   marker_color = '#1caeb0')
layout = go.Layout(title = 'Superficie Total vs Precio (SAN ANTONIO DE PRADO)',
                   xaxis_title = 'Superficie Total',
                   yaxis_title = 'Precio COP')
fig = go.Figure(data = [trace1, trace2], layout =  layout)
iplot(fig)

## Observaciones

##### Primeras Observaciones:
 - Conjunto de datos con un tamaño nada suficiente para ser representativo, y ademas, con outliers.

## Buenos Aires

In [45]:
# Filtering
buenos_aires = df_real.loc[df_real['BUENOS AIRES'] == 1].drop(columns = ['ALTAVISTA', 'ARANJUEZ', 'BELEN', 
                                                                       'BUENOS AIRES','CASTILLA', 'DOCE DE OCTUBRE',
                                                                       'EL POBLADO', 
                                                                       'GUAYABAL', 'LA AMERICA', 'LA CANDELARIA', 
                                                                       'LAURELES', 'PALMITAS', 'POPULAR', 'ROBLEDO', 
                                                                       'MANRIQUE', 'SAN ANTONIO DE PRADO',
                                                                       'SAN CRISTOBAL', 
                                                                       'SAN JAVIER', 'SANTA CRUZ', 'SANTA ELENA',
                                                                       'VILLA HERMOSA'])
buenos_aires.describe()

,delta_time,lon,lat,bedrooms,bathrooms,surface_total,price,Apartamento
count,445.000000,445.000000,445.000000,445.000000,445.000000,445.000000,4.450000e+02,445.000000
mean,87.429213,-75.560526,6.223623,3.804494,2.997753,278.516854,6.795728e+08,0.483146
std,51.156821,0.004819,0.008409,1.571130,1.387685,422.714424,6.374636e+08,0.500278
min,0.000000,-75.566002,6.215319,1.000000,1.000000,12.000000,2.390000e+08,0.000000
25%,49.000000,-75.563004,6.217000,3.000000,2.000000,87.000000,3.100000e+08,0.000000
50%,85.000000,-75.563004,6.219000,3.000000,3.000000,147.000000,4.241650e+08,0.000000
75%,119.000000,-75.559998,6.228309,4.000000,4.000000,270.000000,7.800000e+08,1.000000
max,368.000000,-75.539000,6.244000,10.000000,10.000000,4000.000000,3.500000e+09,1.000000


In [46]:
apto_baires = buenos_aires[buenos_aires['Apartamento'] == 1]
casa_baires = buenos_aires[buenos_aires['Apartamento'] == 0]
print('Numero de apatamentos: ', apto_baires.shape[0])
print('Numero de casas: ', casa_baires.shape[0])

Numero de apatamentos:  215
Numero de casas:  230


In [48]:
trace1 = go.Scatter(x = apto_baires.surface_total, y = apto_baires.price, mode = 'markers', 
                    name = 'Apartamentos', marker_color = '#ffd000')
trace2 = go.Scatter(x = casa_baires.surface_total, y = casa_baires.price, mode = 'markers', name = 'Casas', 
                   marker_color = '#1caeb0')
layout = go.Layout(title = 'Superficie Total vs Precio (BUENOS AIRES)',
                   xaxis_title = 'Superficie Total',
                   yaxis_title = 'Precio COP')
fig = go.Figure(data = [trace1, trace2], layout =  layout)
iplot(fig)

## Observaciones

##### Primeras Observaciones:
 - Caso parecido al de LA CANDELARIA. No se si el conjunto de datos es lo suficientemente grande para ser representativo, sin embargo, se logra ver una agrupacion de datos en la parte inferior izquierda.
 - Outliers por posible mala digitacion de datos

## El Poblado

In [49]:
# Filtering
poblado = df_real.loc[df_real['EL POBLADO'] == 1].drop(columns = ['ALTAVISTA', 'ARANJUEZ', 'BELEN', 
                                                                       'BUENOS AIRES','CASTILLA', 'DOCE DE OCTUBRE',
                                                                       'EL POBLADO', 
                                                                       'GUAYABAL', 'LA AMERICA', 'LA CANDELARIA', 
                                                                       'LAURELES', 'PALMITAS', 'POPULAR', 'ROBLEDO', 
                                                                       'MANRIQUE', 'SAN ANTONIO DE PRADO',
                                                                       'SAN CRISTOBAL', 
                                                                       'SAN JAVIER', 'SANTA CRUZ', 'SANTA ELENA',
                                                                       'VILLA HERMOSA'])
poblado.describe()

,delta_time,lon,lat,bedrooms,bathrooms,surface_total,price,Apartamento
count,2380.000000,2380.000000,2380.000000,2380.000000,2380.000000,2380.000000,2.380000e+03,2380.000000
mean,75.208403,-75.567305,6.202706,3.297059,3.497059,386.276891,8.266809e+08,0.639496
std,66.034582,0.007044,0.012351,1.142437,1.336427,5738.033171,6.008888e+08,0.480248
min,0.000000,-75.582000,6.177000,1.000000,1.000000,28.000000,2.390000e+08,0.000000
25%,28.750000,-75.571999,6.193000,3.000000,2.000000,105.000000,4.500000e+08,0.000000
50%,65.000000,-75.568000,6.203000,3.000000,3.000000,155.000000,6.500000e+08,1.000000
75%,91.000000,-75.563577,6.213000,4.000000,4.000000,250.000000,9.500000e+08,1.000000
max,454.000000,-75.549000,6.230000,12.000000,10.000000,198000.000000,4.500000e+09,1.000000


In [51]:
apto_poblado = poblado[poblado['Apartamento'] == 1]
casa_poblado = poblado[poblado['Apartamento'] == 0]
print('Numero de apatamentos: ', apto_poblado.shape[0])
print('Numero de casas: ', casa_poblado.shape[0])

Numero de apatamentos:  1522
Numero de casas:  858


In [52]:
trace1 = go.Scatter(x = apto_poblado.surface_total, y = apto_poblado.price, mode = 'markers', 
                    name = 'Apartamentos', marker_color = '#ffd000')
trace2 = go.Scatter(x = casa_poblado.surface_total, y = casa_poblado.price, mode = 'markers', name = 'Casas', 
                   marker_color = '#1caeb0')
layout = go.Layout(title = 'Superficie Total vs Precio (EL POBLADO)',
                   xaxis_title = 'Superficie Total',
                   yaxis_title = 'Precio COP')
fig = go.Figure(data = [trace1, trace2], layout =  layout)
iplot(fig)

## Observaciones

##### Primeras Observaciones:
 - Conjunto de datos lo suficientemente grande para ser representativo.
 - No se puede analizar el conjunto de datos debido a un outlier que evidentemente esta mal digitado. 

# Analisis de Bajos de Superficie Total

aranjuez manrique popular san javier

## Aranjuez

In [54]:
# Filtering
aranjuez = df_real.loc[df_real['ARANJUEZ'] == 1].drop(columns = ['ALTAVISTA', 'ARANJUEZ', 'BELEN', 
                                                                       'BUENOS AIRES','CASTILLA', 'DOCE DE OCTUBRE',
                                                                       'EL POBLADO', 
                                                                       'GUAYABAL', 'LA AMERICA', 'LA CANDELARIA', 
                                                                       'LAURELES', 'PALMITAS', 'POPULAR', 'ROBLEDO', 
                                                                       'MANRIQUE', 'SAN ANTONIO DE PRADO',
                                                                       'SAN CRISTOBAL', 
                                                                       'SAN JAVIER', 'SANTA CRUZ', 'SANTA ELENA',
                                                                       'VILLA HERMOSA'])
aranjuez.describe()

,delta_time,lon,lat,bedrooms,bathrooms,surface_total,price,Apartamento
count,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,8.700000e+01,87.000000
mean,68.839080,-75.560863,6.272873,4.068966,2.275862,149.310345,3.375081e+08,0.425287
std,43.110694,0.004711,0.006140,1.538597,0.757745,73.498190,1.015633e+08,0.497253
min,2.000000,-75.569000,6.263905,1.000000,1.000000,55.000000,2.400000e+08,0.000000
25%,32.000000,-75.566002,6.268000,3.000000,2.000000,95.500000,2.700000e+08,0.000000
50%,71.000000,-75.559000,6.271000,4.000000,2.000000,133.000000,3.000000e+08,0.000000
75%,92.000000,-75.556529,6.277954,5.000000,3.000000,160.000000,3.850000e+08,1.000000
max,189.000000,-75.554701,6.286238,9.000000,5.000000,430.000000,9.000000e+08,1.000000


In [55]:
apto_aranjuez = aranjuez[aranjuez['Apartamento'] == 1]
casa_aranjuez = aranjuez[aranjuez['Apartamento'] == 0]
print('Numero de apatamentos: ', apto_aranjuez.shape[0])
print('Numero de casas: ', casa_aranjuez.shape[0])

Numero de apatamentos:  37
Numero de casas:  50


In [57]:
trace1 = go.Scatter(x = apto_aranjuez.surface_total, y = apto_aranjuez.price, mode = 'markers', 
                    name = 'Apartamentos', marker_color = '#ffd000')
trace2 = go.Scatter(x = casa_aranjuez.surface_total, y = casa_aranjuez.price, mode = 'markers', name = 'Casas', 
                   marker_color = '#1caeb0')
layout = go.Layout(title = 'Superficie Total vs Precio (ARANJUEZ)',
                   xaxis_title = 'Superficie Total',
                   yaxis_title = 'Precio COP')
fig = go.Figure(data = [trace1, trace2], layout =  layout)
iplot(fig)

## Observaciones

##### Primeras Observaciones:
 - Conjunto de datos pequeño. Presencia de dos outliers, el del precio, mal catalogado. Conjunto de datos medianamente balanceado.

## Manrique

In [59]:
# Filtering
manrique = df_real.loc[df_real['MANRIQUE'] == 1].drop(columns = ['ALTAVISTA', 'ARANJUEZ', 'BELEN', 
                                                                       'BUENOS AIRES','CASTILLA', 'DOCE DE OCTUBRE',
                                                                       'EL POBLADO', 
                                                                       'GUAYABAL', 'LA AMERICA', 'LA CANDELARIA', 
                                                                       'LAURELES', 'PALMITAS', 'POPULAR', 'ROBLEDO', 
                                                                       'MANRIQUE', 'SAN ANTONIO DE PRADO',
                                                                       'SAN CRISTOBAL', 
                                                                       'SAN JAVIER', 'SANTA CRUZ', 'SANTA ELENA',
                                                                       'VILLA HERMOSA'])
manrique.describe()

,delta_time,lon,lat,bedrooms,bathrooms,surface_total,price,Apartamento
count,51.000000,51.000000,51.000000,51.0,51.000000,51.000000,5.100000e+01,51.000000
mean,74.294118,-75.549476,6.270787,4.0,2.627451,178.490196,4.564902e+08,0.411765
std,63.659184,0.003842,0.006087,1.6,0.958348,103.471421,3.080555e+08,0.497050
min,2.000000,-75.554337,6.263000,1.0,1.000000,62.000000,2.400000e+08,0.000000
25%,24.500000,-75.553081,6.266535,3.0,2.000000,115.500000,2.660000e+08,0.000000
50%,78.000000,-75.550377,6.269000,4.0,2.000000,162.000000,3.500000e+08,0.000000
75%,90.500000,-75.545825,6.274367,5.0,3.000000,215.000000,5.725000e+08,1.000000
max,299.000000,-75.542000,6.284000,10.0,5.000000,670.000000,2.100000e+09,1.000000


In [60]:
apto_manrique = manrique[manrique['Apartamento'] == 1]
casa_manrique = manrique[manrique['Apartamento'] == 0]
print('Numero de apatamentos: ', apto_manrique.shape[0])
print('Numero de casas: ', casa_manrique.shape[0])

Numero de apatamentos:  21
Numero de casas:  30


In [62]:
trace1 = go.Scatter(x = apto_manrique.surface_total, y = apto_manrique.price, mode = 'markers', 
                    name = 'Apartamentos', marker_color = '#ffd000')
trace2 = go.Scatter(x = casa_manrique.surface_total, y = casa_manrique.price, mode = 'markers', name = 'Casas', 
                   marker_color = '#1caeb0')
layout = go.Layout(title = 'Superficie Total vs Precio (MANRIQUE)',
                   xaxis_title = 'Superficie Total',
                   yaxis_title = 'Precio COP')
fig = go.Figure(data = [trace1, trace2], layout =  layout)
iplot(fig)

## Observaciones

##### Primeras Observaciones:
 - Conjunto de datos bastante pequeño, ligeramente agrupados, tres outliers, mal catalogados o casas finca.

## Popular

In [63]:
# Filtering
popular = df_real.loc[df_real['POPULAR'] == 1].drop(columns = ['ALTAVISTA', 'ARANJUEZ', 'BELEN', 
                                                                       'BUENOS AIRES','CASTILLA', 'DOCE DE OCTUBRE',
                                                                       'EL POBLADO', 
                                                                       'GUAYABAL', 'LA AMERICA', 'LA CANDELARIA', 
                                                                       'LAURELES', 'PALMITAS', 'POPULAR', 'ROBLEDO', 
                                                                       'MANRIQUE', 'SAN ANTONIO DE PRADO',
                                                                       'SAN CRISTOBAL', 
                                                                       'SAN JAVIER', 'SANTA CRUZ', 'SANTA ELENA',
                                                                       'VILLA HERMOSA'])
popular.describe()

,delta_time,lon,lat,bedrooms,bathrooms,surface_total,price,Apartamento
count,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,1.100000e+01,11.000000
mean,55.000000,-75.547750,6.291943,4.636364,2.818182,213.727273,4.277273e+08,0.181818
std,41.494578,0.005149,0.004847,2.907670,0.603023,121.399416,2.207302e+08,0.404520
min,8.000000,-75.554352,6.285762,1.000000,2.000000,68.000000,2.650000e+08,0.000000
25%,16.000000,-75.552224,6.287951,3.000000,2.500000,113.500000,2.900000e+08,0.000000
50%,62.000000,-75.548000,6.295000,3.000000,3.000000,164.000000,3.350000e+08,0.000000
75%,77.000000,-75.543000,6.295000,6.500000,3.000000,328.500000,4.150000e+08,0.000000
max,140.000000,-75.542000,6.298000,9.000000,4.000000,356.000000,8.600000e+08,1.000000


In [64]:
apto_popular = popular[popular['Apartamento'] == 1]
casa_popular = popular[popular['Apartamento'] == 0]
print('Numero de apatamentos: ', apto_popular.shape[0])
print('Numero de casas: ', casa_popular.shape[0])

Numero de apatamentos:  2
Numero de casas:  9


In [65]:
trace1 = go.Scatter(x = apto_popular.surface_total, y = apto_popular.price, mode = 'markers', 
                    name = 'Apartamentos', marker_color = '#ffd000')
trace2 = go.Scatter(x = casa_popular.surface_total, y = casa_popular.price, mode = 'markers', name = 'Casas', 
                   marker_color = '#1caeb0')
layout = go.Layout(title = 'Superficie Total vs Precio (POPULAR)',
                   xaxis_title = 'Superficie Total',
                   yaxis_title = 'Precio COP')
fig = go.Figure(data = [trace1, trace2], layout =  layout)
iplot(fig)

## Observaciones

##### Primeras Observaciones:
 - Conjunto de datos para nada representativo, tamaño indudablemente insuficiente y con outliers. NO SIRVE.

## San Javier

In [66]:
# Filtering
san_javier = df_real.loc[df_real['SAN JAVIER'] == 1].drop(columns = ['ALTAVISTA', 'ARANJUEZ', 'BELEN', 
                                                                       'BUENOS AIRES','CASTILLA', 'DOCE DE OCTUBRE',
                                                                       'EL POBLADO', 
                                                                       'GUAYABAL', 'LA AMERICA', 'LA CANDELARIA', 
                                                                       'LAURELES', 'PALMITAS', 'POPULAR', 'ROBLEDO', 
                                                                       'MANRIQUE', 'SAN ANTONIO DE PRADO',
                                                                       'SAN CRISTOBAL', 
                                                                       'SAN JAVIER', 'SANTA CRUZ', 'SANTA ELENA',
                                                                       'VILLA HERMOSA'])
san_javier.describe()

,delta_time,lon,lat,bedrooms,bathrooms,surface_total,price,Apartamento
count,46.000000,46.000000,46.000000,46.000000,46.000000,46.000000,4.600000e+01,46.000000
mean,72.934783,-75.613127,6.256933,3.782609,2.673913,282.456522,4.119130e+08,0.391304
std,64.144768,0.006725,0.005239,1.459270,1.136356,703.284286,3.255756e+08,0.493435
min,4.000000,-75.631000,6.247000,1.000000,1.000000,54.000000,2.400000e+08,0.000000
25%,17.500000,-75.617996,6.254002,3.000000,2.000000,82.000000,2.550000e+08,0.000000
50%,66.000000,-75.615499,6.256500,3.000000,2.000000,143.000000,3.150000e+08,0.000000
75%,97.000000,-75.608251,6.261000,4.750000,3.000000,192.750000,3.987500e+08,1.000000
max,295.000000,-75.601000,6.267296,8.000000,5.000000,4553.000000,2.300000e+09,1.000000


In [67]:
apto_sjavier = san_javier[san_javier['Apartamento'] == 1]
casa_sjavier = san_javier[san_javier['Apartamento'] == 0]
print('Numero de apatamentos: ', apto_sjavier.shape[0])
print('Numero de casas: ', casa_sjavier.shape[0])

Numero de apatamentos:  18
Numero de casas:  28


In [68]:
trace1 = go.Scatter(x = apto_sjavier.surface_total, y = apto_sjavier.price, mode = 'markers', 
                    name = 'Apartamentos', marker_color = '#ffd000')
trace2 = go.Scatter(x = casa_sjavier.surface_total, y = casa_sjavier.price, mode = 'markers', name = 'Casas', 
                   marker_color = '#1caeb0')
layout = go.Layout(title = 'Superficie Total vs Precio (SAN JAVIER)',
                   xaxis_title = 'Superficie Total',
                   yaxis_title = 'Precio COP')
fig = go.Figure(data = [trace1, trace2], layout =  layout)
iplot(fig)

## Observaciones

##### Primeras Observaciones:
 - Conjunto de datos pequeño, con una clara agrupacion y datos outliers, claramente mal digitados. No esta balanceado